In [1]:
import re
from vnpy.trader.constant import Exchange
from vnpy.app.cta_strategy.base import EXCHANGE_SYMBOL_DICT
from datetime import datetime, timedelta

In [36]:
fromDate = datetime.strptime('2012-6-10', '%Y-%m-%d')
toDate = datetime.strptime('2017-12-01', '%Y-%m-%d')

count = 0
while fromDate.date() != toDate.date():
    count += 1
    fromDate = fromDate + timedelta(1)

print(count)

2000


In [16]:
# 当前时间戳
import time
a = int(time.time())
print(a)

1563864223


In [59]:
# 时间戳转时间
timeStamp = 1512136800
date = datetime.fromtimestamp(timeStamp)
print(date)

2017-12-01 22:00:00


In [23]:
# 时间转时间戳
a = '2019-07-20 08:00:00'
b = datetime.strptime(a, '%Y-%m-%d %H:%M:%S')
print(b.timestamp())

1563580800.0


In [3]:
import requests
URL_EXCHANGES = 'https://www.cryptocompare.com/api/data/exchanges'
response = requests.get(URL_EXCHANGES).json()
data = response['Data']
print(data.keys())

dict_keys(['decoin', 'Binance', 'Bitstamp', 'BitTrex', 'OKCoin', 'BTER', 'Kraken', 'Cexio', 'Yacuna', 'Coinbase', 'LocalBitcoins', 'itBit', 'HitBTC', 'BTC38', 'Coinfloor', 'BTCExchange', 'hbus', 'LakeBTC', 'Bit2C', 'CCEX', 'Gatecoin', 'Yunbi', 'WavesDEX', 'CCEDK', 'Cryptopia', 'Exmo', 'Yobit', 'BitBay', 'Korbit', 'BTCMarkets', 'Coincheck', 'QuadrigaCX', 'BitSquare', 'Vaultoro', 'MercadoBitcoin', 'Unocoin', 'Bitso', 'BTCXIndia', 'Paymium', 'TheRockTrading', 'bitFlyer', 'Quoine', 'Liqui', 'Luno', 'CryptoX', 'EtherDelta', 'TuxExchange', 'CHBTC', 'bitFlyerFX', 'LiveCoin', 'Tidex', 'CoinCorner', 'BitMarket', 'Coinone', 'Bleutrade', 'EthexIndia', 'Surbitcoin', 'Bithumb', 'Foxbit', 'Zaif', 'Jubi', 'ViaBTC', 'Novaexchange', 'Lykke', 'Remitano', 'Coinroom', 'Abucoins', 'BXinth', 'HuobiPro', 'OKEX', 'Gateio', 'TrustDEX', 'EXX', 'Kucoin', 'BitGrail', 'BitZ', 'BitFlip', 'TradeSatoshi', 'Coincap', 'ChileBit', 'VBTC', 'Velox', 'Graviex', 'ExtStock', 'BitBank', 'CoinDeal', 'AidosMarket', 'TokenStore'

In [3]:
from pymongo import MongoClient, ASCENDING, DESCENDING
from vnpy.app.cta_strategy.base import TICK_DB_NAME, MINUTE_DB_NAME, DAILY_DB_NAME, MinuteDataBaseName, HourDataBaseName
from datetime import datetime
from vnpy.trader.object import BarData, TickData

In [85]:
# ====== Tick ======
client = MongoClient('localhost', 27017, serverSelectionTimeoutMS = 600)       # Mongo连接
client.server_info()
dbTick = client[TICK_DB_NAME]
symbol_list = ['BTCUSD', 'ETHUSD', 'EOSUSD']
for symbol in symbol_list:
    cl = dbTick[symbol]
    cl.create_index('datetime')         # 添加索引
    start = datetime.strptime('2019-11-18 07:59:50', '%Y-%m-%d %H:%M:%S')
    end = datetime.strptime('2019-11-19 08:00:10', '%Y-%m-%d %H:%M:%S')
    flt = {'datetime': {'$gte':start,
                        '$lte':end}}
    cursor = cl.find({}).sort('datetime', DESCENDING)
    count = 0
    for dic in cursor:
        tick = TickData(gateway_name='', symbol='', exchange=None, datetime=None)
        tick.__dict__ = dic
        print(f'vt_symbol:{tick.vt_symbol}\t{tick.datetime}\t{tick.symbol}\t{tick.last_price}')
        count += 1
        if count >= 1:
            break

vt_symbol:BTCUSD.BYBIT	2020-03-23 04:12:08.448000	BTCUSD	5845.5
vt_symbol:ETHUSD.BYBIT	2020-03-23 08:25:43.646000	ETHUSD	124.85
vt_symbol:EOSUSD.BYBIT	2020-03-23 08:25:43.644000	EOSUSD	2.157


In [86]:
# ====== Minute ======
client = MongoClient('localhost', 27017)
db = client[MINUTE_DB_NAME]
collection = db['BTCUSD.BYBIT']
start = datetime.strptime('2019-12-31 00:00:00', '%Y-%m-%d %H:%M:%S')
end = datetime.strptime('2019-12-20 00:06:00', '%Y-%m-%d %H:%M:%S')
flt = {'datetime': {'$gte':start,
                    '$lte':end}}
cursor = collection.find({}).sort('datetime', DESCENDING)
count = 0
for dic in cursor:
    print(dic['datetime'])
    count += 1
    if count >= 6:
        break

2020-03-23 08:29:00
2020-03-23 08:28:00
2020-03-23 08:27:00
2020-03-23 08:26:00
2020-03-23 08:25:00
2020-03-23 08:24:00


In [89]:
# ====== Daily ======
client = MongoClient('localhost', 27017)
db = client[DAILY_DB_NAME]
symbol_list = ['BTCUSD.BYBIT', 'ETHUSD.BYBIT', 'EOSUSD.BYBIT']
for symbol in symbol_list:
    collection = db[symbol]
    start = datetime.strptime('2019-11-27 00:00:00', '%Y-%m-%d %H:%M:%S')
    end = datetime.strptime('2019-11-27 00:06:00', '%Y-%m-%d %H:%M:%S')
    flt = {'datetime': {'$gte':start,
                        '$lte':end}}
    cursor = collection.find({}).sort('datetime', DESCENDING)
    for dic in cursor:
        symbol = dic['symbol']
        the_datetime = dic['datetime']
        open_price = dic['open_price']
        high_price = dic['high_price']
        low_price = dic['low_price']
        close_price = dic['close_price']
        print(f'{symbol}\t{the_datetime}\t{open_price}\t{high_price}\t{low_price}\t{close_price}')
        break

BTCUSD.BYBIT	2020-03-22 08:00:00	6190.5	6420.0	5735.0	5815.0
ETHUSD.BYBIT	2020-03-22 08:00:00	132.7	137.25	120.75	122.25
EOSUSD.BYBIT	2020-03-22 08:00:00	2.277	2.394	2.114	2.138


In [5]:
symbol_list = ['RB2005', 'HC2005', 'SM2005', 'J2005', 'ZC2005', 'TA2005',
              'RB99', 'HC99', 'SM99', 'J99', 'ZC99', 'TA99']
symbol_list = ['BTCUSD.BYBIT']
client = MongoClient('localhost', 27017)
db = client[DAILY_DB_NAME]
for symbol in symbol_list:
    collection = db[symbol]
    cursor = collection.find({}).sort('datetime', DESCENDING)
    count = 0
    for dic in cursor:
        count += 1
        if count == 1:
            symbol = dic['symbol']
            exchange = dic['exchange']
            the_datetime = dic['datetime']
            open_price = dic['open_price']
            high_price = dic['high_price']
            low_price = dic['low_price']
            close_price = dic['close_price']
            print(f'{symbol}\t{exchange}\t{the_datetime}\t{open_price}\t{high_price}\t{low_price}\t{close_price}')
            break
        
    print('\n')

BTCUSD.BYBIT	None	2019-12-24 08:00:00	7310.0	7429.0	7155.5	7254.0




In [10]:
# ====== 检查Tick质量 ======
client = MongoClient('localhost', 27017, serverSelectionTimeoutMS = 600)       # Mongo连接
client.server_info()
dbTick = client[TICK_DB_NAME]
cl = dbTick['BTCUSD']
cl.create_index('datetime')         # 添加索引
start = datetime.strptime('2019-12-20 07:59:50', '%Y-%m-%d %H:%M:%S')
end = datetime.strptime('2019-12-30 08:00:10', '%Y-%m-%d %H:%M:%S')
flt = {'datetime': {'$gte':start,
                    '$lte':end}}
cursor = cl.find(flt).sort('datetime')
count = 0
for dic in cursor:
    tick = TickData(gateway_name='', symbol='', exchange=None, datetime=None)
    tick.__dict__ = dic
    if tick.last_price == 0:
        print(f'vt_symbol:{tick.vt_symbol}\t{tick.datetime}\t{tick.symbol}\t{tick.last_price}\t{tick.ask_price_1}\t{tick.bid_price_1}')
    count += 1
print(count)

vt_symbol:BTCUSD.BYBIT	2019-12-24 00:19:04.628000	BTCUSD	0	7669.0	7668.5
vt_symbol:BTCUSD.BYBIT	2019-12-24 00:19:04.647000	BTCUSD	0	7669.0	7668.5
vt_symbol:BTCUSD.BYBIT	2019-12-24 00:19:04.768000	BTCUSD	0	7669.0	7668.5
vt_symbol:BTCUSD.BYBIT	2019-12-24 00:19:04.848000	BTCUSD	0	7669.0	7668.5
vt_symbol:BTCUSD.BYBIT	2019-12-24 00:19:04.868000	BTCUSD	0	7669.0	7668.5
vt_symbol:BTCUSD.BYBIT	2019-12-24 00:19:04.887000	BTCUSD	0	7669.0	7668.5
vt_symbol:BTCUSD.BYBIT	2019-12-24 00:19:04.908000	BTCUSD	0	7669.0	7668.5
vt_symbol:BTCUSD.BYBIT	2019-12-24 00:19:04.968000	BTCUSD	0	7669.0	7668.5
vt_symbol:BTCUSD.BYBIT	2019-12-24 00:19:05.088000	BTCUSD	0	7669.0	7668.5
vt_symbol:BTCUSD.BYBIT	2019-12-24 00:19:05.247000	BTCUSD	0	7669.0	7668.5
vt_symbol:BTCUSD.BYBIT	2019-12-24 00:19:05.408000	BTCUSD	0	7669.0	7668.5
vt_symbol:BTCUSD.BYBIT	2019-12-24 00:19:05.528000	BTCUSD	0	7669.0	7668.5
vt_symbol:BTCUSD.BYBIT	2019-12-24 00:19:05.927000	BTCUSD	0	7669.0	7668.5
vt_symbol:BTCUSD.BYBIT	2019-12-24 00:19:06.028000	B

In [2]:
a = 'a'
if a:
    print('yes')
else:
    print('no')

yes
